This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'

from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
SUFFIX = '20201201' #'20190701'
master_catalogue = Table.read("../../dmu32/dmu32_Bootes/data/Bootes_{}_cigale.fits".format(SUFFIX))

In [ ]:
for col in master_catalogue.colnames:
    #Remove Brown estimated total mags
    if ('_lbc_' in col) or ('_suprime_' in col):
        master_catalogue.remove_column(col)

In [3]:
len(master_catalogue)

3398098

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
good = {}
for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
    good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
                                   ~master_catalogue['flag_{}'.format(band)])
    good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
                              master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [5]:
combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
print("Number of good sources: {}".format(np.sum(combined_good)))

Number of good sources: 45482


In [7]:
# Only sources with at least two optical and at least two near infrared detections
optnir = ((master_catalogue['flag_optnir_det'] == 3) 
          | (master_catalogue['flag_optnir_det'] == 7))

# Main catalogue for CIGALE

In [8]:
best_catalogue = master_catalogue[combined_good].copy()
#best_catalogue = master_catalogue.copy()

In [9]:
#best_catalogue.remove_column("redshift")
#best_catalogue["zspec"].name = "redshift"

In [10]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [11]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [12]:
# Convertion to CIGALE format
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 3686 sources, the band lbc_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2723 sources, the band mosaic_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1718 sources, the band 90prime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1702 sources, the band gpc1_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 440 sources, the band gpc1_r should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 353 so

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [13]:
#u_bands = ["lbc_u"]
g_bands = [            "90prime_g", "gpc1_g"]
r_bands = ["mosaic_r", "90prime_r", "gpc1_r"]
i_bands = ["mosaic_i",              "gpc1_i"]
z_bands = ["mosaic_z", "90prime_z", "gpc1_z" ]
y_bands = ["gpc1_y"]

j_bands = ["newfirm_j", "ukidss_j"]
h_bands = ["newfirm_h"]
k_bands = ["newfirm_k",           "tifkam_ks"]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [14]:
#remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(k_bands)

In [15]:
best_catalogue.write("data_tmp/Bootes_cigale_best_extcor_{}.fits".format(SUFFIX), overwrite=True)

## All optnir objects with photoz

In [16]:
best_catalogue = master_catalogue[optnir].copy()


In [17]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]


In [18]:
print("Number of optnir sources with z-phot: {}".format(len(best_catalogue)))


Number of optnir sources with z-phot: 1558627


In [19]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)


In [20]:
# Convertion to CIGALE format
os.environ['LOG_LEVEL'] = 'INFO'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 320042 sources, the band lbc_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 259818 sources, the band mosaic_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 191828 sources, the band 90prime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 190631 sources, the band gpc1_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 49061 sources, the band gpc1_r should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:

In [21]:

#remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(k_bands)

In [22]:
best_catalogue.write("data_tmp/Bootes_cigale_optnir_extcor_{}.fits".format(SUFFIX), overwrite=True)

# Catalogue using spectroscopic redshift

In [23]:
best_catalogue = master_catalogue[optnir].copy()

In [24]:
best_catalogue.remove_column("redshift")
best_catalogue["zspec"].name = "redshift"

In [25]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [26]:
print("Number of sources with z-spec: {}".format(len(best_catalogue)))

Number of sources with z-spec: 55069


In [27]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [28]:
# Convertion to CIGALE format
os.environ['LOG_LEVEL'] = 'INFO'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 30896 sources, the band lbc_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 30764 sources, the band mosaic_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 30624 sources, the band 90prime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 30621 sources, the band gpc1_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 30478 sources, the band gpc1_r should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 

In [29]:
#remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(k_bands)

In [30]:
best_catalogue.write("data_tmp/Bootes_cigale_optnir_extcor_zspec_{}.fits".format(SUFFIX), overwrite=True)